In [1]:
!pip install mdbtools-dev
!apt-get install mdbtools
!pip install pandas pyodbc


ERROR: Could not find a version that satisfies the requirement mdbtools-dev (from versions: none)
ERROR: No matching distribution found for mdbtools-dev
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following additional packages will be installed:
  libmdb3 libmdbsql3
Suggested packages:
  bash-completion
The following NEW packages will be installed:
  libmdb3 libmdbsql3 mdbtools
0 upgraded, 3 newly installed, 0 to remove and 49 not upgraded.
Need to get 136 kB of archives.
After this operation, 486 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu jammy/universe amd64 libmdb3 amd64 1.0.0+dfsg-1 [68.9 kB]
Get:2 http://archive.ubuntu.com/ubuntu jammy/universe amd64 libmdbsql3 amd64 1.0.0+dfsg-1 [18.8 kB]
Get:3 http://archive.ubuntu.com/ubuntu jammy/universe amd64 mdbtools amd64 1.0.0+dfsg-1 [48.3 kB]
Fetched 136 kB in 0s (729 kB/s)
Selecting previously unselected package libmdb3:amd64.
(Reading databas

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
import sys
sys.path.append('/content/drive/MyDrive/Dev/py2/')

import pandas as pd
from google.colab import files
import os
import subprocess
from io import StringIO  # StringIOを正しくインポート
import model.utility.k_analysis_util as k_a_util


In [9]:

# 使用例
table_name = 'PredictorVariables'  # テーブル名を指定

file_list = [f'kjdb_prediction_cache_{year}.accdb' for year in range(2000, 2025)]

k_a_util.prepare_pd_on_notebook()
print(len(file_list))

df_statistics=[]
for file_name in file_list:
    filepath = f'/content/drive/MyDrive/Dev/Kjvan/{file_name}'
    df_summary = k_a_util.access_file_statistics(filepath, table_name)
    columns_to_drop = [ 'key_horse_id', 'desc_horse_no', 'upd', 'upd']
    df = df_summary.drop(columns=columns_to_drop)
    df_statistics.append(df)
    print(file_name)
    print(df)
    print("")

25
kjdb_prediction_cache_2000.accdb
        obj_rank   cats_plc  cats_year   cats_mnt  cats_evtd   cats_rpc   cats_wtr   cats_trk   cats_bct   cats_bcd   cats_cat  cats_cls2  cats_cls3  cats_cls4  cats_cls5  cats_clsy  cats_rule  \
count 71831.0000 71831.0000 71831.0000 71831.0000 71831.0000 71831.0000 71831.0000 71831.0000 71831.0000 71831.0000 71831.0000 71831.0000 71831.0000 71831.0000 71831.0000 71831.0000 71831.0000   
mean      6.7714     6.8872     0.0000     5.7831     3.9261     3.0750     1.6194     0.0000     0.4054     1.0996     1.7042     0.1338     1.1984     1.2930     1.3954     1.8457     9.2477   
std       4.0168     3.0899     0.0000     3.3240     4.7271     2.1536     0.7913     0.0000     0.6703     1.0885     1.5326     0.5943     1.5928     1.9851     2.1113     1.7315    10.9818   
min       0.0000     0.0000     0.0000     0.0000     0.0000     0.0000     0.0000     0.0000     0.0000     0.0000     0.0000     0.0000     0.0000     0.0000     0.0000     0.000

In [ ]:
import pandas as pd
import numpy as np

def summarize_dataframes(df1, df2):
    """
    2つのDataFrameの統計量を計算し、まとめたDataFrameを返す。

    Args:
        df1: 1つ目のDataFrame
        df2: 2つ目のDataFrame

    Returns:
        統計量をまとめたDataFrame。エラーが発生した場合はNoneを返す。
    """

    try:
        # インデックスを揃えるために転置
        df1_t = df1.transpose()
        df2_t = df2.transpose()

        # 数値データのみを抽出
        df1_numeric = df1_t.apply(pd.to_numeric, errors='coerce')
        df2_numeric = df2_t.apply(pd.to_numeric, errors='coerce')

        # 統計量を計算
        summary = pd.DataFrame({
            'mean': (df1_numeric.mean(axis=1) + df2_numeric.mean(axis=1)) / 2,
            'std': ((df1_numeric.std(axis=1)**2 + df2_numeric.std(axis=1)**2)/2)**0.5, #分散の平均からstdを計算
            'min': pd.concat([df1_numeric.min(axis=1), df2_numeric.min(axis=1)], axis=1).min(axis=1)
        })
        return summary

    except Exception as e:
        print(f"エラーが発生しました: {e}")
        return None

# サンプルのDataFrameを作成 (実際のデータに合わせて変更してください)
data1 = {'obj_rank': [6.7714, 4.0168, 0.0000], 'cats_plc': [6.8872, 3.0899, 0.0000]}
data2 = {'obj_rank': [6.8978, 4.0882, 0.0000], 'cats_plc': [6.9564, 3.1208, 0.0000]}
index = ['mean', 'std', 'min']

df1 = pd.DataFrame(data1, index=index)
df2 = pd.DataFrame(data2, index=index)

# 統計量を計算
summary_df = summarize_dataframes(df1, df2)

if summary_df is not None:
    print(summary_df)
